In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/My Drive/dataset/"

In [ ]:
import nltk
nltk.download('words')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import words
import random
import torch.optim as optim
import numpy as np
import math
import sys
import warnings
warnings.filterwarnings("ignore")
from textblob import TextBlob
def correct_word(word):
  if word=="\n":
    return "\n"
  elif word=="mot":
    return "not"
  elif word=="mo":
    return "no"
  elif word =="j":
    return "i"
  else:
    return str(TextBlob(word).correct())

In [ ]:
class Anastasia(nn.Module):
    def __init__(self, input_size,vocab_size):
        super(Anastasia, self).__init__()
        self.lstm = nn.LSTM(input_size,256,batch_first=True)
        self.linear1 = nn.Linear(256,512)
        self.dropout1 = nn.Dropout(0.4)
        self.linear2 = nn.Linear(512,1024)
        self.linear3 = nn.Linear(1024,1024)
        self.dropout2 = nn.Dropout(0.75)
        self.out = nn.Linear(1024,vocab_size)
    def forward(self, input):
        output, _ = self.lstm(input)
        output = torch.tanh(output)
        output = self.linear1(output)
        output = F.leaky_relu(output)
        output = self.dropout1(output)
        output = self.linear2(output)
        output = F.relu(output)
        output = self.linear3(output)
        output = torch.tanh(output)
        output = self.dropout2(output)
        output = self.out(output)
        output = torch.reshape(output,(input.shape[0],vocab_len))
        return output
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]


In [ ]:
def predict(model,dataX,vocab_len,length=100,value=""):
    if len(value)<50:
      start = np.random.randint(0, len(dataX)-1)
      pattern = dataX[start]
      head = X_son_head[start]
      head = np.reshape(head,(1,50,1))
    else:
      val = value[:50]
      pattern = [char_to_int[i] for i in val]
      """value = np.array(list(value))
      pattern = np.reshape(value,(1,50,1))"""
    word_out = ""
    print("Given Input:\n",''.join([int_to_char[value] for value in pattern]))
    count_nl = 0
    print("Prediction:")
    for i in range(length):
        x = np.reshape(pattern[-50:], (1, 1, 50))
        x = x / float(vocab_len)
        x = torch.tensor(x)
        #x = np.array([np_utils.to_categorical(j,num_classes=vocab_len) for j in x])
        prediction = F.softmax(model(x))
        index = torch.argmax(prediction).item()
        prediction = np.reshape(prediction.detach().numpy(),(vocab_len))
        if int_to_char[index]=="\n" and char_to_int['\n'] in pattern[-40:]:
          #sys.stdout.write("1")
          while int_to_char[index]=="\n":
            index = np.random.choice(vocab_len, 1, p=prediction)[0]
        if ((pattern[-1]==char_to_int[" "] or pattern[-1]==char_to_int["\n"] or pattern[-1]==char_to_int["t"]) and random.random()<0.25) or int_to_char[index]=="-":
          #sys.stdout.write("2")
          index = np.random.choice(vocab_len, 1, p=prediction)[0]
          while int_to_char[index]=="\n" or int_to_char[index]==" " or int_to_char[index]=="-":
            index = np.random.choice(vocab_len, 1, p=prediction)[0]
        result = int_to_char[index]
        if result == " " and char_to_int['\n'] not in pattern[-40:len(pattern)]:
          #sys.stdout.write("3")
          #print(pattern[10:len(pattern)],end='')
          result = '\n'
          index = char_to_int['\n']
        if result == '\n':
          count_nl+=1
          if count_nl>13:
            break
        seq_in = [int_to_char[value] for value in pattern]
        #sys.stdout.write(result)
        pattern.append(index)
        word_out += result
        word_out = word_out.replace("\n"," \n ")
        wordslist = word_out.split()
        #print(wordslist)
        if len(wordslist)>2:
          if wordslist[-1] == wordslist[-2] or (len(wordslist[-1])>9 and wordslist[-1] not in set(words.words())):
            lastword = wordslist.pop(-1)
            word_out = word_out[:-len(lastword)]
            pattern = pattern[:-len(lastword)].copy()
            i -=len(lastword)
        #pattern = pattern[1:len(pattern)]
    out = [int_to_char[i].replace("\n"," \n ") for i in pattern]
    #print(out)
    out = "".join(out)
    out = [correct_word(i) for i in out.split(" ")]
    out =  " ".join(out).replace("\n ","\n")
    print(out)
    print("\n")

In [ ]:
def Punctuation(string): 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~”“'''
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
    return string

In [ ]:
file2 = open(base_path+"text.txt","r")
data = file2.read().lower()
sonnets_data = data.split("\n")
print(sonnets_data)

In [ ]:
chars = sorted(list(set(data)))
chars = sorted(chars)
vocab_len = len(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
print("Vocab Length:",vocab_len)
print(int_to_char)
print([(i,c) for i,c in enumerate(sonnets_data)])

In [ ]:
def random_sonnets(sonnets_data,seq_len=50):
  sonnets = []
  seq_length = seq_len
  dataX = []
  dataX_Char = []
  X_son_head = []
  dataY = []
  for i in range(2,len(sonnets_data),17):
      string = "\n".join(sonnets_data[i:i+14])
      sonnets.append(string)
  random.shuffle(sonnets)
  print(sonnets)
  for son in sonnets:
      son_len = len(son)
      son_head = [char_to_int[i] for i in son[0:seq_length]]
      for j in range(0, son_len - seq_length, 1):
        seq_in = son[j:j + seq_length]
        seq_out = son[j + seq_length]
        X_son_head.append(son_head)
        dataX_Char.append([char for char in seq_in])
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
  pattern_len = len(dataX)
  print(pattern_len)
  X = np.reshape(dataX, (pattern_len, 1, seq_length))
  X = torch.tensor(X / float(vocab_len)).double()
  X_son_head = np.reshape(X_son_head, (pattern_len, 1, seq_length))
  X_son_head = torch.tensor(X_son_head/ float(vocab_len)).double()
  y = torch.tensor(dataY)
  return dataX,dataY,X,y,X_son_head
dataX,dataY,X,y,X_son_head = random_sonnets(sonnets_data)

In [ ]:
print(y.shape)
print(X.shape)
seq_length = 50
size = y.shape[0]
print(size)

In [ ]:
BATCH_SIZE = 32
train_split = 0.7
dataX,dataY,X,y,X_son_head = random_sonnets(sonnets_data)
train_x = X[:int(train_split*size)]
train_y = y[:int(train_split*size)]
val_x = X[int(train_split*size):]
val_y = y[int(train_split*size):]
val_size = val_y.shape[0]
filepath_load = base_path+"pytorch_anastasia_attempt1_50_changeddropout_punc.pth"
filepath_save = base_path+"pytorch_anastasia_attempt1_50_changeddropout_punc.pth"
current_val_loss = float('inf')
current_val_acc = 0
model = Anastasia(seq_length,vocab_len).double()
print(model)
model.load_state_dict(torch.load(filepath_load))
epochs = 500
criterion = nn.CrossEntropyLoss()
batches = int(train_x.size()[0]/BATCH_SIZE)
optimizer = optim.SGD(model.parameters(),lr=0.1)
lambdalr = lambda epoch: 0.1/(1+np.exp(epoch/80-2))+0.001
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambdalr])
epoch_loss_list = []
val_loss_list = []
accuracy_list = []
val_accuracy_list = []
patience = 400
checker = 0
for epoch in range(epochs):
  if epoch%10 == 0:
    dataX,dataY,X,y,X_son_head = random_sonnets(sonnets_data)
    train_x = X[:int(train_split*size)]
    train_y = y[:int(train_split*size)]
    val_x = X[int(train_split*size):]
    val_y = y[int(train_split*size):]
    val_size = val_y.shape[0]
  if checker >= patience:
    print("Val Loss didn't decrease in last 5 epochs")
    break
  epoch_loss = 0
  correct = 0
  permutation = torch.randperm(train_x.size()[0])
  for i in range(0,train_x.size()[0], BATCH_SIZE):
    step = int(i/BATCH_SIZE)
    optimizer.zero_grad()
    indices = permutation[i:i+BATCH_SIZE]
    batch_x, batch_y,batch_h = train_x[indices], train_y[indices],X_son_head[indices]
    yhat = model(batch_x)
    loss = criterion(yhat, batch_y.long())
    epoch_loss += loss.item()
    _,predicted=torch.max(yhat,1)
    correct += (predicted==batch_y).sum().item()/BATCH_SIZE
    if i>0:
      print('\rEpoch: '+str(epoch+1)+' Batch '+str(int(i))+' Loss: '+str(epoch_loss/i*BATCH_SIZE)+' Accuracy: '+str(correct/i*BATCH_SIZE)+' Steps left',batches-step,end="")
    loss.backward()
    optimizer.step()
  val_yhat = model(val_x)
  _,predicted_val=torch.max(val_yhat,1)
  loss_val = criterion(val_yhat, val_y.long())
  correct_val = (predicted_val==val_y).sum().item()/val_size
  print("\rEpoch",epoch+1,"completed. Loss", epoch_loss/batches,"Accuracy:",correct/batches,"Val Loss", loss_val.item(),"Val Accuracy:",correct_val)
  scheduler.step()
  if loss_val < current_val_loss and correct_val > current_val_acc*0.995:
    checker = 0
    current_val_loss = loss_val
    current_val_acc = correct_val
    if loss_val < 4:
      torch.save(model.state_dict(),filepath_save)
      print("Val Loss decreased and the model state dict has been saved")
  else:
    #loss_val.backward()
    #optimizer.step()
    if loss_val*0.98 > current_val_loss:
      checker += 1
  predict(model,dataX,vocab_len,100000,"")
  if (epoch)%5 == 0:
    val_loss_list.append(loss_val.item())
    val_accuracy_list.append(correct_val)
    epoch_loss_list.append(epoch_loss/batches)
    accuracy_list.append(correct/batches)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epoch_loss_list)
plt.plot(val_loss_list)
plt.show()
plt.plot(val_accuracy_list)
plt.plot(accuracy_list)
plt.show()

In [ ]:
filepath_load = base_path+"pytorch_anastasia_attempt1_50_changeddropout_punc.pth"
model = Anastasia(seq_length,vocab_len).double()
model.load_state_dict(torch.load(filepath_load))
predict_input(model,vocab_len,14,"from the fairest creatures we desire increase,\nwhile beauty may die in the")

In [ ]:
def predict_input(model,vocab_len,newcount=14,value=""):
    val = value[:50]
    pattern = [char_to_int[i] for i in val]
    n_val = char_to_int['\n']
    word_out = ""
    print("Given Input:\n",''.join([int_to_char[value] for value in pattern]))
    count_nl = pattern.count(n_val)
    print("Prediction:")
    i = 0
    while count_nl!=newcount:
        x = np.reshape(pattern[-50:], (1, 1, 50))
        x = x / float(vocab_len)
        x = torch.tensor(x)
        #x = np.array([np_utils.to_categorical(j,num_classes=vocab_len) for j in x])
        prediction = F.softmax(model(x))
        index = torch.argmax(prediction).item()
        prediction = np.reshape(prediction.detach().numpy(),(vocab_len))
        if int_to_char[index]=="\n" and char_to_int['\n'] in pattern[-40:]:
          #sys.stdout.write("1")
          while int_to_char[index]=="\n":
            index = np.random.choice(vocab_len, 1, p=prediction)[0]
        if ((pattern[-1]==char_to_int[" "] or pattern[-1]==char_to_int["\n"] or pattern[-1]==char_to_int["t"]) and random.random()<0.25) or int_to_char[index]=="-":
          #sys.stdout.write("2")
          index = np.random.choice(vocab_len, 1, p=prediction)[0]
          while int_to_char[index]=="\n" or int_to_char[index]==" " or int_to_char[index]=="-":
            index = np.random.choice(vocab_len, 1, p=prediction)[0]
        result = int_to_char[index]
        if result == " " and char_to_int['\n'] not in pattern[-40:len(pattern)]:
          #sys.stdout.write("3")
          #print(pattern[10:len(pattern)],end='')
          result = '\n'
          index = char_to_int['\n']
        if result == '\n':
          count_nl+=1
        seq_in = [int_to_char[value] for value in pattern]
        #sys.stdout.write(result)
        pattern.append(index)
        word_out += result
        word_out = word_out.replace("\n"," \n ")
        wordslist = word_out.split()
        #print(wordslist)
        if len(wordslist)>2:
          if wordslist[-1] == wordslist[-2] or len(wordslist[-1])>9:
            lastword = wordslist.pop(-1)
            word_out = word_out[:-len(lastword)]
            pattern = pattern[:-len(lastword)].copy()
            i -=len(lastword)
        #pattern = pattern[1:len(pattern)]
        i+=1
    out = [int_to_char[i].replace("\n"," \n ") for i in pattern]
    #print(out)
    out = "".join(out)
    out = [correct_word(i) for i in out.split(" ")]
    out =  " ".join(out).replace("\n ","\n")
    print(out)
    print("\n")